### Statistiques concernant les cararactéristiques des incidents ayant fait des victimes : comment expliquer le nombre de victimes en fonction de l'incident ?

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)
import plotly.express as px

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
url="https://drive.google.com/file/d/1LO4i_-TkSmxnR0dgO5_lVLZ35FtGTNzA/view?usp=drive_link"
url="https://drive.google.com/uc?export=download&confirm=1&id=" + url.split("/")[-2]
total_db = pd.read_csv(url)
total_db.sample(5)

,Unnamed: 0,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,geometry,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop
1120,1120,Wolfe,Kentucky,KY,152,28459,Wolfe,"POLYGON ((-83.707974 37.716463, -83.652357 37....",21237.0,13.0,10.1,11.0,9.6,8.3,9.5,25657.0,25768.0,27196.0,28443.0,29326.0,30144.0,7.282,7.255,7.247,7.203,7.264,7.215,2940.0,2925.0,2541.0,2524.0,2483.0,2304.0,11.8,10.7,11.9,11.8,7.8,7.6,1.25,1578331,2492184,3218371,3676847,4641480,4912441,0.285877,0.684890
2587,2587,Cottle,Texas,TX,27326,29949,Cottle,"POLYGON ((-100.518088 34.003886, -100.517991 3...",48101.0,4.6,7.7,6.3,4.8,4.1,4.4,32292.0,32722.0,35602.0,34605.0,36201.0,38304.0,1.442,1.427,1.437,1.398,1.373,1.375,258.0,264.0,227.0,238.0,240.0,201.0,16.1,15.2,15.2,16.2,16.7,15.3,2.45,1633278,1465992,1574266,1721726,1549434,1571632,0.144082,0.833619
1602,1602,Taney,Missouri,MO,154,947,Taney,"POLYGON ((-93.314903 36.51426, -93.313648 36.5...",29213.0,8.8,7.9,6.1,5.5,4.4,12.5,34860.0,35731.0,39320.0,38268.0,43110.0,40887.0,53.298,53.861,54.282,54.663,55.176,55.817,8720.0,8207.0,7919.0,7476.0,7390.0,7141.0,18.6,17.9,18.3,18.4,17.8,18.4,0.40,567009,517063,584875,632319,524973,496184,0.186632,0.779520
2992,2992,Carroll,Virginia,VA,27330,33809,Carroll,"POLYGON ((-81.03405599999999 36.811603, -81.03...",51035.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.00,498426,419764,462997,528369,520827,476760,0.188195,0.784058
7,7,Calhoun,Alabama,AL,27282,27343,Calhoun,"POLYGON ((-86.146225 33.702176, -86.145772 33....",1015.0,8.2,6.7,6.7,6.0,4.2,5.0,37898.0,41123.0,42091.0,41778.0,45937.0,45400.0,116.528,115.991,115.550,115.036,114.746,114.298,25918.0,25709.0,24247.0,22183.0,20782.0,19434.0,16.1,16.9,17.6,17.7,17.9,18.0,2.15,563880,621305,737509,616947,477345,474294,0.278559,0.692397


In [63]:
gun_violence_db = pd.read_csv('df_gun_violence.csv')
gun_violence_db.sample(5)

,Unnamed: 0,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
82289,82733,395149,2015-07-30,New York,Brooklyn,Willoughby Street,0,0,http://www.gunviolencearchive.org/incident/395149,http://www.nbcnewyork.com/news/local/NYC-Armed...,False,8.0,NaN,NaN,"['Non-Shooting Incident', 'Home Invasion', 'Ho...",40.6920,NaN,-73.9800,NaN,"Brownsville, hi, robbed at gunpoint, no inj",{0: '41'},{0: 'Adult 18+'},"{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}",NaN,NaN,"{0: 'Unharmed', 1: 'Unharmed', 2: 'Unharmed', ...","{0: 'Victim', 1: 'Subject-Suspect', 2: 'Subjec...",NaN,57.0,25.0,0,0,0,1,0,NaN,NaN,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
229367,228793,1034137,2018-01-23,North Carolina,East Bend,Butner Mill Road and NC-67,0,0,http://www.gunviolencearchive.org/incident/103...,http://myfox8.com/2018/01/23/armed-and-dangero...,False,5.0,{0: 'Unknown'},{0: 'Unknown'},"['Shots Fired - No Injuries', 'Officer Involve...",36.1988,NaN,-80.5235,1.0,NaN,{0: '34'},{0: 'Adult 18+'},{0: 'Male'},0::Benji Alton East,NaN,"{0: 'Unharmed, Arrested'}",{0: 'Subject-Suspect'},NaN,73.0,31.0,0,0,0,0,0,Unknown,NaN,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0
52478,52673,279817,2015-01-03,New York,Brooklyn,2919 Surf Avenue,0,1,http://www.gunviolencearchive.org/incident/279817,http://nypost.com/2015/01/18/daily-blotter-409/,False,8.0,NaN,NaN,"['Shot - Wounded/Injured', 'Institution/Group/...",40.5729,NaN,-73.9950,NaN,"Coney Island, bodega, perps opened fire, vic h...","{0: '18', 1: '23'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+'}","{0: 'Male', 1: 'Male', 2: 'Male'}",1::Luis Cintron,NaN,"{0: 'Injured', 1: 'Unharmed, Arrested', 2: 'Un...","{0: 'Victim', 1: 'Subject-Suspect', 2: 'Subjec...",['http://7online.com/news/police-looking-for-g...,46.0,23.0,0,0,0,0,0,NaN,NaN,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,1,1
49929,48315,263430,2014-12-14,Tennessee,Nashville,825 Long Hunter Court,1,1,http://www.gunviolencearchive.org/incident/263430,http://wate.com/2014/12/15/metro-officer-shot-...,False,5.0,{0: 'Unknown'},{0: 'Handgun'},"['Shot - Wounded/Injured', 'Shot - Dead (murde...",36.0950,NaN,-86.6389,1.0,"Nashboro Village area at Hunters Trace, teen r...","{0: '42', 1: '16'}","{0: 'Adult 18+', 1: 'Teen 12-17'}","{0: 'Male', 1: 'Male'}",0::Officer John Downs||1::Xavier McDonald,NaN,"{0: 'Injured', 1: 'Killed'}","{0: 'Victim', 1: 'Subject-Suspect'}",['http://wate.com/2014/12/15/metro-officer-sho...,52.0,19.0,1,0,0,0,1,NaN,NaN,0,1,1,1,0,0,0,1,1,0,0,0,1,0,0,0,0,0
119798,118034,535896,2016-04-05,Illinois,Peoria,2700 block of West Trewyn Avenue,0,1,http://www.gunviolencearchive.org/incident/535896,http://www.pjstar.com/news/20160405/shooting-i...,False,17.0,NaN,NaN,NaN,40.6641,NaN,-89.6364,NaN,"Male wounded, leg, unclear;",NaN,NaN,{0: 'Male'},NaN,NaN,{0: 'Injured'},{0: 'Victim'},NaN,92.0,46.0,0,0,0,0,0,NaN,NaN,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [5]:
from shapely import wkt
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
total_db = total_db.dropna(subset=['geometry'])
total_db['geometry'] = total_db['geometry'].apply(wkt_loads)

In [6]:
total_db = total_db.loc[total_db['Etat'] != 'Alaska']
total_db = total_db.loc[total_db['Etat'] != 'Hawaii']

gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Alaska']
gun_violence_db = gun_violence_db.loc[gun_violence_db['state'] != 'Hawaii']


In [7]:
total_db_geo = (
    gpd.GeoDataFrame(total_db)
    .set_geometry('geometry')
    .set_crs('EPSG:4269')
)

In [8]:
#changement de valeur pour Richmond, Virginia car valeur aberrante de 8.755 alors qu'il y avait 204 214	habitants en 2010
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2013'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2014'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2015'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2016'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2017'] = 204.214 
total_db_geo.loc[total_db_geo['id_county'] == 30306, 'resident_pop_year_2018'] = 204.214

In [9]:
total_pop=total_db_geo.groupby('Etat').agg({'resident_pop_year_2015':'sum'}).reset_index()
total_pop.rename(columns={'resident_pop_year_2015':'pop_state_2015'}, inplace=True)
total_pop

,Etat,pop_state_2015
0,Alabama,4854.803
1,Arizona,6832.810
2,Arkansas,2979.732
3,California,38904.296
4,Colorado,5454.328
5,Connecticut,3588.561
6,Delaware,942.065
7,District of Columbia,677.014
8,Florida,20219.111
9,Georgia,10183.353


In [10]:
total_db_geo=total_db_geo.merge(total_pop, on ='Etat', how='left')

In [64]:
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(gun_violence_db['longitude'], gun_violence_db['latitude'])]
gun_violence_geo = gpd.GeoDataFrame(gun_violence_db, geometry=geometry, crs='EPSG:4326')

gun_violence_geo = gun_violence_geo.set_crs('EPSG:4326')
gun_violence_geo = gun_violence_geo[gun_violence_geo.geometry.x <= -50]

In [12]:
total_db_geo=total_db_geo.to_crs('EPSG:4326')

In [65]:
merge_geo = gpd.sjoin(gun_violence_geo, total_db_geo, predicate='within')

In [66]:
geom_county=total_db_geo[['id_county','geometry']]
geom_county.head()

,id_county,geometry
0,27336,"POLYGON ((-86.92119 32.65754, -86.92035 32.658..."
1,27337,"POLYGON ((-88.02858 30.22676, -88.02399 30.230..."
2,27338,"POLYGON ((-85.74803 31.61918, -85.74543 31.618..."
3,27339,"POLYGON ((-87.42194 33.00338, -87.31854 33.006..."
4,27340,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."


In [67]:
merge_geo2=pd.merge(merge_geo, geom_county, on='id_county')
merge_geo2.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict,geometry_x,index_right,Unnamed: 0_right,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,pop_state_2015,geometry_y
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Mass Shooting (4+ ...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/01/01/4-...,NaN,NaN,0,0,0,0,0,NaN,NaN,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,POINT (-79.85590 40.34670),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Shot - Dead (murde...",40.4555,Diverse Banquet Hall,-79.8970,NaN,NaN,"{0: '21', 1: '27'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Fema...",0::Jasmine Morris||1::Demetrius Broadnax,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/10/20/ho...,NaN,NaN,1,0,0,0,1,NaN,NaN,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,POINT (-79.89700 40.45550),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at The Waterf

In [68]:
merge_geo = merge_geo2.rename(columns={'geometry_x': 'point_geometry', 'geometry_y': 'polygon_geometry'})
merge_geo.head()

,Unnamed: 0_left,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type_dict,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,indic_suicide,indic_defens,indic_school,indic_home_inv,indic_accident,gun_type,relation,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict,point_geometry,index_right,Unnamed: 0_right,Nom,Etat,Code_Etat,id_Etat,id_county,NAME,fips,unemp_rate_year_2013,unemp_rate_year_2014,unemp_rate_year_2015,unemp_rate_year_2016,unemp_rate_year_2017,unemp_rate_year_2018,med_h_income_year_2013,med_h_income_year_2014,med_h_income_year_2015,med_h_income_year_2016,med_h_income_year_2017,med_h_income_year_2018,resident_pop_year_2013,resident_pop_year_2014,resident_pop_year_2015,resident_pop_year_2016,resident_pop_year_2017,resident_pop_year_2018,snap_beneficiaries_year_2013,snap_beneficiaries_year_2014,snap_beneficiaries_year_2015,snap_beneficiaries_year_2016,snap_beneficiaries_year_2017,snap_beneficiaries_year_2018,bachelors_deg_year_2013,bachelors_deg_year_2014,bachelors_deg_year_2015,bachelors_deg_year_2016,bachelors_deg_year_2017,bachelors_deg_year_2018,score_legis,bchecks_2013,bchecks_2014,bchecks_2015,bchecks_2016,bchecks_2017,bchecks_2018,per_dem,per_gop,pop_state_2015,polygon_geometry
0,100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Mass Shooting (4+ ...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",0::Julian Sims,NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/01/01/4-...,NaN,NaN,0,0,0,0,0,NaN,NaN,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,1,POINT (-79.85590 40.34670),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
1,104811,481268,2013-10-21,Pennsylvania,Pittsburgh (Homewood),7200 block of Kelly Street,2,5,http://www.gunviolencearchive.org/incident/481268,http://www.wtae.com/news/local/allegheny/two-p...,False,14.0,NaN,NaN,"['Shot - Wounded/Injured', 'Shot - Dead (murde...",40.4555,Diverse Banquet Hall,-79.8970,NaN,NaN,"{0: '21', 1: '27'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Fema...",0::Jasmine Morris||1::Demetrius Broadnax,NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",['http://pittsburgh.cbslocal.com/2013/10/20/ho...,NaN,NaN,1,0,0,0,1,NaN,NaN,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,POINT (-79.89700 40.45550),2215,2258,Allegheny,Pennsylvania,PA,27320,29615,Allegheny,42003.0,5.4,4.7,4.5,4.6,4.2,4.9,51400.0,52385.0,54412.0,56099.0,58547.0,59881.0,1233.999,1233.417,1229.342,1226.893,1220.423,1217.508,160102.0,160100.0,160552.0,154950.0,152435.0,146826.0,35.9,36.9,37.8,39.1,40.0,40.7,4.0,1044731,899241,993429,1143220,1062651,1021943,0.564416,0.40034,12789.838,"POLYGON ((-80.36078 40.47760, -80.34227 40.490..."
2,678,94128,2014-01-08,Pennsylvania,Homestead,Steak ‘n Shake at 

In [69]:
base_totale=merge_geo.copy()

## Après cet import compliqué, on peut passer aux statistiques

In [18]:
print("I. Les dates. \nNous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.")

I. Les dates. 
Nous allons commencer cette analyse des conséquences des incidents en termes de victimes (blessés et morts) par l'étude des effets potentiels de la date sur le nombre et la gravité des incidents.


In [19]:
base_totale['date']=pd.to_datetime(base_totale['date'])

In [20]:
grouped_data = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()

fig1 = px.bar(
    grouped_data,
    x=grouped_data['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par an",
    color_discrete_map={"n_injured": "blue", "n_killed": "red"},
)

fig1.update_layout(barmode='group')  # Display bars side by side

fig1.show()


In [21]:
print("Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également. Ainsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.")

Ce graphique nous montre que, non seulement le nombre d'incidents par arme à feu augmente sur nos 4 années complètes considérées, mais que le nombre de victimes (blessés et tués) augmente également. Ainsi, les conséquences de ces incidents touchent de plus en plus d'individus au fur et à mesure des années.


In [22]:
grouped_data2 = base_totale.groupby(base_totale['date'].dt.year).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()

grouped_data2=grouped_data2.loc[grouped_data2['date'].isin([2014,2015,2016,2017])]

fig2 = px.bar(
    grouped_data2,
    x=grouped_data2['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Moyenne de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par an",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig2.update_layout(barmode='group')  # Display bars side by side

fig2.show()

In [23]:
print("Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.")

Ce second graphique nous montre que l'augmentation des victimes repérée précédemment est liée quasiment exclusivement à l'augementation du nombre d'incident, car le nombre de victime par incident reste relativement stable sur les 4 années observées, mis à part une légèrement augementation du nombre de blessés moyen par incident entre 2014 et 2015.


In [24]:
grouped_data3 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig3 = px.bar(
    grouped_data3,
    x=grouped_data3['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "Mois", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes d'incidents par arme à feu par mois (cumulé de 2014 à 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig3.update_layout(barmode='group')  # Display bars side by side

fig3.show()



In [25]:
grouped_data4 = base_totale.loc[base_totale['date'].dt.year.isin([2014,2015,2016,2017])].groupby(
    base_totale['date'].dt.month).agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig4 = px.bar(
    grouped_data4,
    x=grouped_data4['date'],
    y=['n_injured', 'n_killed'],
    labels={"date": "année", "value": "Nombre moyen de victimes par incident", "variable": "Type de victime"},
    title="Moyenne de victimes d'incident par arme à feu par mois (entre 2014 et 2017)",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig4.update_layout(barmode='group')  # Display bars side by side

fig4.show()

In [26]:
print("On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. \nDe plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. \nIl semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.")

On retrouve ici la même variation que pour le nombre d'incidents : il y a plus de blessés et de morts par arme à feu pendant l'été aux Etats-Unis que pendant l'hiver. 
De plus, même si le nombre de morts par incident reste stable autour de 0,25, le nombre moyen de victimes par incident varie de 0.46 en Février à 0.55 en Juillet. 
Il semble donc exister un effet de la saisonnalité sur le nombre d'incidents mais également sur leur gravité et leurs conséquences en termes de victimes.


In [27]:
print("II. Les Etats. \nCette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans une partie suivante. \nIntéressons nous d'abord aux conséquences des incidents par arme à feu dans chaque Etat.")

II. Les Etats. 
Cette analyse des incidents en fonction des Etats sera complétée par une visualisation cartographique dans une partie suivante. 
Intéressons nous d'abord aux conséquences des incidents par arme à feu dans chaque Etat.


In [28]:
grouped_data5 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig5 = px.bar(
    grouped_data5,
    x=grouped_data5['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig5.update_layout(barmode='group')  # Display bars side by side

fig5.show()

In [41]:
print("Ce graphique mets en évidence les Etats dans lesquels les victimes sont les plus nombreuses. Les Etats dans lesquels il y a eu le plus de blessés sont l'Illinois, le Maryland, la Virginie et le Missouri alors que ceux dans lesquels il y a eu le plus de morts sont la Californie, le Missouri, le Maryland, la Virginie et le Texas. \nOn voit déjà ici que selon l'endroit où un incident par arme à feu se déroule, il n'a pas les mêmes conséquences. Il y a donc des caractéristiques spécifiques aux Etats qui doivent expliquer ces différences. \nOn peut également remarquer que ces Etats ne sont pas exactement les mêmes que ceux dans lesquels il y a le plus d'incidents par arme à feu, un incident ne semble donc pas avoir la même gravité en termes de conséquences partout.")

Ce graphique mets en évidence les Etats dans lesquels les victimes sont les plus nombreuses. Les Etats dans lesquels il y a eu le plus de blessés sont l'Illinois, le Maryland, la Virginie et le Missouri alors que ceux dans lesquels il y a eu le plus de morts sont la Californie, le Missouri, le Maryland, la Virginie et le Texas. 
On voit déjà ici que selon l'endroit où un incident par arme à feu se déroule, il n'a pas les mêmes conséquences. Il y a donc des caractéristiques spécifiques aux Etats qui doivent expliquer ces différences. 
On peut également remarquer que ces Etats ne sont pas exactement les mêmes que ceux dans lesquels il y a le plus d'incidents par arme à feu, un incident ne semble donc pas avoir la même gravité en termes de conséquences partout.


In [39]:
grouped_data6 = base_totale.groupby('state').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig6 = px.bar(
    grouped_data6,
    x=grouped_data6['state'],
    y=['n_injured', 'n_killed'],
    labels={"state": "Etat", "value": "Moyenne du nombre de victimes par incident", "variable": "Type de victime"},
    title="Nombre moyen de victimes par incident par arme à feu par Etat",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},
)

fig6.update_layout(barmode='group')  # Display bars side by side

fig6.show()

In [47]:
print("Ce graphique nous permet de mettre en évidence des différences en terme de nombre moyen de victimes par incident. \nAinsi, dans l'Illinois, les incidents font en moyenne plus de blessés que dans les autres Etats (presque 0,8 par incident) alors qu'ils font relativement peu de morts (0,18). \nAu contraire, dans certains Etats tels que l'Arizona, les incidents font un nombre moyen de blessés (0.46) mais la moyenne de morts par incident est très élevée (quasiment 0,47 par incident). \nLes taux de mortalité et de 'blessure' d'un incident varient donc beaucoup d'un Etat à l'autre, et ne sont pas nécessairement corrélés. Dans certains Etats, les incidents font peu de morts et peu de blessés (Maine, New Hampshire), \ndans d'autres ils font beaucoup de blessés et de morts (Arizona), dans d'autres encore ils font beaucoup de blessés et peu de morts (Illinois, Maryland) et finalement ils peuvent même faire plus de morts que de blessés en moyenne (Wyoming, Montana, Idaho).")

Ce graphique nous permet de mettre en évidence des différences en terme de nombre moyen de victimes par incident. 
Ainsi, dans l'Illinois, les incidents font en moyenne plus de blessés que dans les autres Etats (presque 0,8 par incident) alors qu'ils font relativement peu de morts (0,18). 
Au contraire, dans certains Etats tels que l'Arizona, les incidents font un nombre moyen de blessés (0.46) mais la moyenne de morts par incident est très élevée (quasiment 0,47 par incident). 
Les taux de mortalité et de 'blessure' d'un incident varient donc beaucoup d'un Etat à l'autre, et ne sont pas nécessairement corrélés. Dans certains Etats, les incidents font peu de morts et peu de blessés (Maine, New Hampshire), 
dans d'autres ils font beaucoup de blessés et de morts (Arizona), dans d'autres encore ils font beaucoup de blessés et peu de morts (Illinois, Maryland) et finalement ils peuvent même faire plus de morts que de blessés en moyenne (Wyoming, Montana, Idaho).


In [44]:
grouped_data7 = base_totale.groupby('state').agg({
    'n_injured': 'sum',
    'n_killed': 'sum',
    'pop_state_2015':'first'
}).reset_index()
grouped_data7['injured_per_100K']=grouped_data7["n_injured"]/grouped_data7['pop_state_2015']*100
grouped_data7['killed_per_100K']=grouped_data7["n_killed"]/grouped_data7['pop_state_2015']*100

fig7 = px.bar(
    grouped_data7,
    x=grouped_data7['state'],
    y=['injured_per_100K', 'killed_per_100K'],
    labels={"state": "Etat", "value": "nombre de victimes pour 100 000 habitants", "variable": "Type de victime"},
    title="Nombre de victimes d'incident par arme à feu pour 100 000 habitants par Etat",
    color_discrete_map={"injured_per_100K": "firebrick", "killed_per_100K": "steelblue"},
)

fig7.update_layout(barmode='group')  # Display bars side by side

fig7.show()

In [52]:
print("Ce graphique mets en évidence les Etats ayant le plus de blessés et de tués lors d'incidents par arme à feu pour 100 000 habitants. \nL'Etat District of Columbia est ici un outlier, alors qu'il ne l'était pas dans les autres mesures des conséquences des incidents, ce qui pourrait indiquer une erreur ou simplement une très forte concentration de victimes en comparaison à la taille de l'Etat. \nOn retrouve les Etats du Maryland, Missouri, Virginie et Illinois comme ceux dans lesquels les incidents par arme à feu font le plus de blessés et de morts. \nEn dehors de ces 11 Etats dans lesquels les incidents par arme à feu ont fait plus de 50 blessés pour 100 000  habitants entre 2013 et 2018, les autres sont tous relativement similaires et les incidents qui s'y déroulent ont provoqué entre 5 et 35 blessés pour 100 000 habitants.")

Ce graphique mets en évidence les Etats ayant le plus de blessés et de tués lors d'incidents par arme à feu pour 100 000 habitants. 
L'Etat District of Columbia est ici un outlier, alors qu'il ne l'était pas dans les autres mesures des conséquences des incidents, ce qui pourrait indiquer une erreur ou simplement une très forte concentration de victimes en comparaison à la taille de l'Etat. 
On retrouve les Etats du Maryland, Missouri, Virginie et Illinois comme ceux dans lesquels les incidents par arme à feu font le plus de blessés et de morts. 
En dehors de ces 11 Etats dans lesquels les incidents par arme à feu ont fait plus de 50 blessés pour 100 000  habitants entre 2013 et 2018, les autres sont tous relativement similaires et les incidents qui s'y déroulent ont provoqué entre 5 et 35 blessés pour 100 000 habitants.


In [57]:
grouped_data8 = base_totale.groupby('state').agg({
    'n_killed': 'mean',
    'indic_suicide':'mean'
}).reset_index()


fig8 = px.bar(
    grouped_data8,
    x=grouped_data8['state'],
    y=['n_killed', 'indic_suicide'],
    labels={"state": "Etat", "value": "Moyenne du nb de morts par incident, taux de suicide", "variable": "Décès"},
    title="Comparaison du nombre moyen de morts par incident et du taux de suicide parmi les incidents, par Etat",
    color_discrete_map={"n_killed": "firebrick", "indic_suicide": "steelblue"},
)

fig8.update_layout(barmode='group')  # Display bars side by side

fig8.show()

In [59]:
print(f"La proportion de suicide parmi les incidents varie largement d'un Etat à l'autre, probablement en raison de caractéristiques socio-démographiques et économiques de ces Etats. On voit que cette proportion est la plus élevée dans l'Arizona (0.28), le Nevada, le Texas, l'Utah et l'Idaho. \nCette proportion est fortement corrélée au nombre de morts ({base_totale['indic_suicide'].corr(base_totale['n_killed'])}), ce qui est logique car la réalisation d'un suicide implique au moins un mort, mais elle est également négativement corrélée au nombre de blessés ({base_totale['indic_suicide'].corr(base_totale['n_injured'])}), \nce qui pourrait expliquer, en partie, pourquoi ces Etats précédemment mentionnés font partie de ceux dans lesquels le nombre de blessés et le nombre de morts par arme à feu sont relativement proches, au contraires d'Etats dans lesquels le nombre de blessés est largement supérieur à celui des morts.")

La proportion de suicide parmi les incidents varie largement d'un Etat à l'autre, probablement en raison de caractéristiques socio-démographiques et économiques de ces Etats. On voit que cette proportion est la plus élevée dans l'Arizona (0.28), le Nevada, le Texas, l'Utah et l'Idaho. 
Cette proportion est fortement corrélée au nombre de morts (0.6276074308002891), ce qui est logique car la réalisation d'un suicide implique au moins un mort, mais elle est également négativement corrélée au nombre de blessés (-0.025296986883618654), 
ce qui pourrait expliquer, en partie, pourquoi ces Etats précédemment mentionnés font partie de ceux dans lesquels le nombre de blessés et le nombre de morts par arme à feu sont relativement proches, au contraires d'Etats dans lesquels le nombre de blessés est largement supérieur à celui des morts.


In [60]:
print("III. Type d'armes. \nIntéressons désormais aux conséquences d'utiliser un certain type d'arme lors d'un incident par arme à feu.")

III. Type d'armes. 
Intéressons désormais aux conséquences d'utiliser un certain type d'arme lors d'un incident par arme à feu.


In [74]:
grouped_data9 = base_totale.groupby('gun_type').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig9 = px.bar(
    grouped_data9,
    x=grouped_data9['gun_type'],
    y=['n_injured', 'n_killed'],
    labels={"gun_type": "Arme", "value": "Nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes en fonction de l'arme à feu",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"},log_y=True
)

fig9.update_layout(barmode='group')  # Display bars side by side

fig9.show()

In [75]:
print("Ce graphique, en échelle logarithmique pour la lisibilité, nous montre que, de manière générale, les armes utilisées font un nombre similaire de blessés et de morts. Il n'y a donc pas d'arme qui ne fasse que des blessés mais pas de mort et inversement. \nDe plus, les quatre types d'armes faisant le plus de victimes sont les mêmes que celles présentes dans le plus d'incidents (Handgun, Rifle, Shotgun, 9mm).")

Ce graphique, en échelle logarithmique pour la lisibilité, nous montre que, de manière générale, les armes utilisées font un nombre similaire de blessés et de morts. Il n'y a donc pas d'arme qui ne fasse que des blessés mais pas de mort et inversement. 
De plus, les quatre types d'armes faisant le plus de victimes sont les mêmes que celles présentes dans le plus d'incidents (Handgun, Rifle, Shotgun, 9mm).


In [81]:
print("IV. Caractéristiques du suspect.")

IV. Caractéristiques du suspect.


In [80]:
grouped_data10 = base_totale.groupby('young_sus').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig10 = px.bar(
    grouped_data10,
    x=grouped_data10['young_sus'],
    y=['n_injured', 'n_killed'],
    labels={"young_sus": "Suspect entre 18 et 25 ans (1)", "value": "Moyenne du nombre de victimes", "variable": "Type de victime"},
    title="Nombre moyen de victimes en fonction de l'âge du suspect",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig10.update_layout(barmode='group')  # Display bars side by side

fig10.show()

In [86]:
print("Lorsque le suspect est jeune, entre 18 et 25 ans, il provoque en moyenne plus de morts lors d'un incident par arme à feu que lorsqu'il appartient à une autre catégorie d'âge (0.28 contre 0.24). \nNéanmoins, un suspect entre 18 et 25 ans fait, en moyenne, moins de blessés qu'un autre type de suspect (0.38 contre 0.53). ")

Lorsque le suspect est jeune, entre 18 et 25 ans, il provoque en moyenne plus de morts lors d'un incident par arme à feu que lorsqu'il appartient à une autre catégorie d'âge (0.28 contre 0.24). 
Néanmoins, un suspect entre 18 et 25 ans fait, en moyenne, moins de blessés qu'un autre type de suspect (0.38 contre 0.53). 


In [97]:
print("V. Circonstances de l'incident par arme à feu")

V. Circonstances de l'incident par arme à feu


In [87]:
grouped_data11 = base_totale.groupby('relation').agg({
    'n_injured': 'mean',
    'n_killed': 'mean'
}).reset_index()


fig11 = px.bar(
    grouped_data11,
    x=grouped_data11['relation'],
    y=['n_injured', 'n_killed'],
    labels={"relation": "Relation entre suspect(s) et victime(s)", "value": "Moyenne du nombre de victimes", "variable": "Type de victime"},
    title="Nombre moyen de victimes en fonction de la relation du suspect aux victimes",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig11.update_layout(barmode='group')  # Display bars side by side

fig11.show()

In [90]:
print("Ce graphique mets en évidence de fortes disparités de conséquences selon les différents types d'incidents par arme à feu, ici caractérisés par les différentes relation du suspect aux victimes. \nEn effet, on remarque que le type d'incident provoquant, en moyenne, le plus de blessés et de morts est la tuerie de masse. Lorsque le suspect connaît les victimes, celles-ci ont plus de chances de mourir que d'être blessées (en moyenne presque 3 morts mais 1 blessé) alors que quand les victimes sont inconnues, celles-ci ont plus de chances d'être blessées que tuées (en moyenne 4 blessés mais 1.4 morts). \nLe deuxième type d'incident par arme à feu provoquant le plus de morts sont les incidents dans le cadre familial. Lorsque la victime et un(e) (ex-)partnaire du suspect, il y a en moyenne 0.7 morts (et 0.3 blessés) et lorsqu'elle fait partie de la famille du suspect, il y a en moyenne 0.8 morts (et 0.4 blessés). \nAinsi, les violences conjugales et intrafamiliales font partie des violences par arme à feu les plus meurtrières, avant les violences entre gangs, par exemple. \nAprès les tueries de masse, les incidents par arme à feu provoquant le plus de blessés sont les violences de gangs (0.8 blessés et 0.47 morts en moyenne) puis les vols à main armée (0.7 blessés et 0.2 morts en moyenne).")

Ce graphique mets en évidence de fortes disparités de conséquences selon les différents types d'incidents par arme à feu, ici caractérisés par les différentes relation du suspect aux victimes. 
En effet, on remarque que le type d'incident provoquant, en moyenne, le plus de blessés et de morts est la tuerie de masse. Lorsque le suspect connaît les victimes, celles-ci ont plus de chances de mourir que d'être blessées (en moyenne presque 3 morts mais 1 blessé) alors que quand les victimes sont inconnues, celles-ci ont plus de chances d'être blessées que tuées (en moyenne 4 blessés mais 1.4 morts). 
Le deuxième type d'incident par arme à feu provoquant le plus de morts sont les incidents dans le cadre familial. Lorsque la victime et un(e) (ex-)partnaire du suspect, il y a en moyenne 0.7 morts (et 0.3 blessés) et lorsqu'elle fait partie de la famille du suspect, il y a en moyenne 0.8 morts (et 0.4 blessés). 
Ainsi, les violences conjugales et intrafamiliales font partie des violences par ar

In [92]:
grouped_data12 = base_totale.groupby('relation').agg({
    'n_injured': 'sum',
    'n_killed': 'sum'
}).reset_index()


fig12 = px.bar(
    grouped_data12,
    x=grouped_data12['relation'],
    y=['n_injured', 'n_killed'],
    labels={"relation": "Relation entre suspect(s) et victime(s)", "value": "nombre de victimes", "variable": "Type de victime"},
    title="Nombre de victimes en fonction de la relation du suspect aux victimes",
    color_discrete_map={"n_injured": "firebrick", "n_killed": "steelblue"}
)

fig12.update_layout(barmode='group')  # Display bars side by side

fig12.show()

In [96]:
print(f"Ce graphique reprenant la même classification des types d'incidents par arme à feu que le précédent permet de se rendre compte des conséquences de ces incidents à l'échelle agrégée plutôt qu'en moyenne. \nLe type d'incidents provoquant le plus de blessés est alors le vol à main armé ({3535/grouped_data12['n_injured'].sum()*100}% des blessés dont la relation avec le suspect est précisée), notamment en raison de leur fréquence très élevée. \nLes violences conjugales et intrafamiliales sont le type d'incident par arme à feu le plus meurtrier ({(3009+2556)/grouped_data12['n_killed'].sum()*100}% des morts dont on connaît la relation au suspect).")

Ce graphique reprenant la même classification des types d'incidents par arme à feu que le précédent permet de se rendre compte des conséquences de ces incidents à l'échelle agrégée plutôt qu'en moyenne. 
Le type d'incidents provoquant le plus de blessés est alors le vol à main armé (40.56690383291255% des blessés dont la relation avec le suspect est précisée), notamment en raison de leur fréquence très élevée. 
Les violences conjugales et intrafamiliales sont le type d'incident par arme à feu le plus meurtrier (65.76459465847317% des morts dont on connaît la relation au suspect).
